<div>
    <img src="https://storage.googleapis.com/kaggle-datasets-images/916586/1553068/ddd9373754b16217a54a513f0d94628a/dataset-cover.png" />
</div>

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier

from tpot import TPOTClassifier, TPOTRegressor
from tpot.config import classifier_config_dict

from deap.gp import Primitive

import os
os.chdir('/kaggle/input/evolution/')
!pip install -qq logger
from evolution import NeuroEvolution

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

<h1 id="dataset" style="color:#a05d40; background:#e8e5dc; border:0.5px dotted;"> 
    <center>Dataset
        <a class="anchor-link" href="#dataset" target="_self">¶</a>
    </center>
</h1>

## Load data

In [ ]:
path = '/kaggle/input/fetal-health-classification/fetal_health.csv'
df = pd.read_csv(path)
df.head()

## Describe data

In [ ]:
df.describe()

## Correlation graph

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(df.corr(), annot=True, fmt='.0%')

## Feature Engineering

In [ ]:
X=df.drop(["fetal_health"],axis=1)
y=df["fetal_health"]

In [ ]:
col_names = list(X.columns)
s_scaler = preprocessing.StandardScaler()
X_df= s_scaler.fit_transform(X)
X_df = pd.DataFrame(X_df, columns=col_names)   
X_df.describe().T

## Split training/testing data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_df, y, test_size=0.25, random_state=23)

<h1 id="class" style="color:#a05d40; background:#e8e5dc; border:0.5px dotted;"> 
    <center>Classification
        <a class="anchor-link" href="#class" target="_self">¶</a>
    </center>
</h1>

## Loading TPOT with XGBoost

In [ ]:
classifier_config_dict['xgboost.XGBClassifier'] = {
    'n_estimators': [100],
    'max_depth': range(1, 11),
    'learning_rate': [1e-3, 1e-2, 1e-1, 0.5, 1.],
    'subsample': np.arange(0.05, 1.01, 0.05),
    'min_child_weight': range(1, 21),
    'n_jobs': [1],
    'verbosity': [0]
}
        
# for a quick test
tpot = TPOTClassifier(generations=2, population_size=10, verbosity=2,
                      config_dict=classifier_config_dict)
tpot.fit(X_train, y_train)

## Fit XGBoost with best parameters

In [ ]:
xgb = XGBClassifier(tpot._optimized_pipeline)
xgb.fit(X_train, y_train)

## Analyze results

In [ ]:
preds = xgb.predict(X_test)
acc = accuracy_score(y_test, preds)
print("Accuracy: {:1.3f}".format(acc))

<h1 id="inference" style="color:#a05d40; background:#e8e5dc; border:0.5px dotted;"> 
    <center>Inference
        <a class="anchor-link" href="#inference" target="_self">¶</a>
    </center>
</h1>

## Transform labels to one hot encodings

In [ ]:
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

## Set the parameters for genetic hyperparameter tunning

In [ ]:
params = {
    "epochs": [25, 75, 100],
    "batch_size": [8, 16, 32],
    "n_layers": [1, 2, 3],
    "n_neurons": [8, 16, 32],
    "dropout": [0.0],
    "optimizers": ["nadam", "adam", "adadelta", "sgd"],
    "activations": ["relu", "sigmoid", "tanh"],
    "last_layer_activations": ["tanh", "sigmoid", "relu", "softmax"],
    "losses": ["mean_absolute_error", "mean_squared_error",
               "mean_squared_logarithmic_error"],
    "metrics": ["accuracy"]
}

## Use of NeuroEvolution to find best parameters

In [ ]:
search = NeuroEvolution(generations=2, population=10, params=params)

search.evolve(X_train, y_train, X_test, y_test)

## Use best parameters to train a DNN

In [ ]:
n_layers = search.best_params.network['n_layers']
n_neurons = search.best_params.network['n_neurons']
activation = search.best_params.network['activations']
last_activation = search.best_params.network['last_layer_activations']

model = tf.keras.Sequential()
for n in range(n_layers):
    model.add(Dense(n_neurons, activation=activation))
model.add(Dense(y_train.shape[1], activation=last_activation))

## Model build and summary

In [ ]:
model.build(X_train.shape)
model.summary()

## Compile and fit the model

In [ ]:
epochs = search.best_params.network['epochs']
batch_size = search.best_params.network['batch_size']
optimizer = search.best_params.network['optimizers']
loss = search.best_params.network['losses']
metrics = [search.best_params.network['metrics']]

In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), verbose=0)

## Analyze results

In [ ]:
_ = plt.figure(figsize=(14,8))
_ = plt.plot(history.history['accuracy'], label="Accuracy")
_ = plt.plot(history.history['loss'], label="Loss")
plt.legend()
plt.show()

In [ ]:
preds = model.predict(X_test)
acc = accuracy_score(np.argmax(preds, axis=1), np.argmax(y_test.values, axis=1))
print("Accuracy: {:1.3f}".format(acc))